In [ ]:
import os
import numpy as np
from nanowire.optics.simulate import Simulator
from nanowire.optics.postprocess import Simulation
from nanowire.optics.utils.utils import setup_sim
from nanowire.optics.utils.config import Config
from nanowire.optics.utils.geometry import get_layers
import scipy.constants as consts
import scipy.integrate as intg
import IPython.display as disp
import matplotlib.pyplot as plt
import time
from itertools import product
plt.style.use(['presentation'])
%load_ext autoreload 
%autoreload 2
%load_ext ipycache

In [ ]:
def integrate(arr, xvals, yvals):
    x_integral = intg.trapz(arr, x=xvals, axis=0)
    y_integral = intg.trapz(x_integral, x=yvals, axis=0)
    return y_integral

In [ ]:
ITO_locs = np.linspace(.501, .8, 4, endpoint=False)
NW_locs = np.linspace(.8, .8+1.3, 4, endpoint=False)
Sub_locs = np.linspace(.8+1.3, .8+1.3+1, 4, endpoint=False)
z_vals = np.concatenate((ITO_locs, NW_locs, Sub_locs))
num_points = np.arange(50, 550, 50)
numbasis = np.arange(200, 550, 50)
pairs = list(zip(num_points, num_points))
print(list(pairs))

In [ ]:
%%cache InPlaneResultsWeismann.pkl results times
results = np.zeros((len(numbasis), len(pairs), z_vals.shape[0]))
times = np.zeros_like(results)
conf = Config('InPlaneSampleTest.yml')
freq = conf[('Simulation', 'params', 'frequency')]
for k, basis in enumerate(numbasis):
    print('#'*25)
    print('Computing with {} basis terms'.format(basis))
    conf[('Simulation', 'params', 'numbasis')] = basis
    for i, (xpts, ypts) in enumerate(pairs):
        print('-'*25)
        print('Computing with {} points per edge'.format(xpts))
        conf[('Simulation', 'x_samples')] = xpts
        conf[('Simulation', 'y_samples')] = ypts
        sim = Simulator(conf)
        sim.setup()
        layers = get_layers(sim)
        for j, z in enumerate(z_vals):
            layer_obj = [layer for layer in layers.values() if layer.start <= z <= layer.end].pop()
            n_mat, k_mat = layer_obj.get_nk_matrix(freq)
            start = time.time()
            Ex, Ey, Ez, Hx, Hy, Hz = sim.compute_fields_on_plane(z, sim.xsamps, sim.ysamps)
            end = time.time()
            diff = end - start
            normEsq = np.absolute(Ex)**2 + np.absolute(Ey)**2 + np.absolute(Ez)**2
            res = integrate(n_mat*k_mat*normEsq, sim.X, sim.Y)
            results[k, i, j] = res
            times[k, i, j] = diff
print('Done!')

# Convergence w.r.t Sampling Points, Fixed Basis

## N = 200

In [ ]:
fig, axes = plt.subplots(4, 3, figsize=(16, 13), sharex=True)
axes = axes.flatten()
total_pts = np.array([tup[0] for tup in pairs])
for i, z in enumerate(z_vals):
    rel_diff = np.array(
        [100*np.abs(results[-1,-1,i]-results[-1,j,i])/results[-1,-1,i] for j in range(len(results[-1, :, i]))])
    #axes[i].plot(total_pts, results[0, :, i], marker='o', label="z = {:.3f}".format(z))
    axes[i].plot(total_pts, rel_diff, marker='o', label="z = {:.3f}".format(z))
    axes[i].legend(loc='best')
    #pdiff = 100*abs(results[0, 0, i] - results[-1, -1, i])/results[0, -1, i]
    #print('Percent Diff = {}'.format(pdiff))
for ax in axes[-3:]:
    ax.set_xlabel("Number of pts per edge")
for ax in axes[0::3]:
    ax.set_ylabel("Percent Difference")
plt.tight_layout()
plt.show()

## N = 500

In [ ]:
fig, axes = plt.subplots(4, 3, figsize=(16, 13), sharex=True)
axes = axes.flatten()
total_pts = np.array([tup[0] for tup in pairs])
for i, z in enumerate(z_vals):
    rel_diff = np.array(
        [100*np.abs(results[-1,-1,i]-results[-1,j,i])/results[-1,-1,i] for j in range(len(results[-1, :, i]))])
    #axes[i].plot(total_pts, results[-1, :, i], marker='o', label="z = {:.3f}".format(z))
    axes[i].plot(total_pts, rel_diff, marker='o', label="z = {:.3f}".format(z))
    axes[i].legend(loc='best')
    #pdiff = 100*abs(results[-1, 1, i] - results[-1, -1, i])/results[-1, -1, i]
    #print('Percent Diff = {}'.format(pdiff))
for ax in axes[-3:]:
    ax.set_xlabel("Number of pts per edge")
for ax in axes[0::3]:
    ax.set_ylabel("Integral Val")
plt.tight_layout()
plt.show()

# Convergence w.r.t Number of Basis Terms, Converged Sampling

In [ ]:
cols = 3
rows = int(np.ceil(len(z_vals))/3)
fig, axes = plt.subplots(rows, cols, figsize=(16, 13), sharex=True)
axes = axes.flatten()
total_pts = np.array([tup[0] for tup in pairs])
for i, z in enumerate(z_vals):
    rel_diff = np.array(
        [100*np.abs(results[-1,2,i]-results[j,2,i])/results[-1,2,i] for j in range(len(results[:, 2, i]))])
    #axes[i].plot(numbasis, results[:, -1, i], marker='o', label="z = {:.3f}".format(z))
    axes[i].plot(numbasis, rel_diff, marker='o', label="z = {:.3f}".format(z))
    axes[i].legend(loc='best')
    #pdiff = 100*abs(results[-2, -1, i] - results[-1, -1, i])/results[-1, -1, i]
    #print('Percent Diff = {} at z = {}'.format(pdiff, z))
for ax in axes[-3:]:
    ax.set_xlabel("Number of Basis Terms")
for ax in axes[0::3]:
    ax.set_ylabel("Integral Val")
plt.tight_layout()
plt.show()

# Convergence w.r.t Number of Basis Terms, Most Dense Sampling

In [ ]:
cols = 3
rows = int(np.ceil(len(z_vals))/3)
fig, axes = plt.subplots(rows, cols, figsize=(16, 13), sharex=True)
axes = axes.flatten()
total_pts = np.array([tup[0] for tup in pairs])
for i, z in enumerate(z_vals):
    rel_diff = np.array(
        [100*np.abs(results[-1,-1,i]-results[j,-1,i])/results[-1,-1,i] for j in range(len(results[:, -1, i]))])
    #axes[i].plot(numbasis, results[:, -1, i], marker='o', label="z = {:.3f}".format(z))
    axes[i].plot(numbasis, rel_diff, marker='o', label="z = {:.3f}".format(z))
    axes[i].legend(loc='best')
    #pdiff = 100*abs(results[-2, -1, i] - results[-1, -1, i])/results[-1, -1, i]
    #print('Percent Diff = {} at z = {}'.format(pdiff, z))
for ax in axes[-3:]:
    ax.set_xlabel("Number of Basis Terms")
for ax in axes[0::3]:
    ax.set_ylabel("Integral Val")
plt.tight_layout()
plt.show()

# Run Time w.r.t Number of Sampling Points, 500 Basis

In [ ]:
cols = 3
rows = int(np.ceil(len(z_vals)/cols))
fig, axes = plt.subplots(rows, cols, figsize=(14, 10), sharex=True, sharey=True)
axes = axes.flatten()
total_pts = np.array([tup[0] for tup in pairs])
for i, z in enumerate(z_vals):
    axes[i].plot(total_pts, times[-1, :, i], marker='o', label="z = {:.3f}".format(z))
    axes[i].legend(loc='best')
for ax in axes[-3:]:
    ax.set_xlabel("Number of pts per edge")
for ax in axes[0::3]:
    ax.set_ylabel("Run (seconds)")
plt.tight_layout()
plt.show()

# Run Time w.r.t Z Location, Fixed Basis

In [ ]:
cols = 3
rows = int(np.ceil(len(pairs)/cols))
fig, axes = plt.subplots(rows, cols, figsize=(16, 13), sharex=True, sharey=True)
axes = axes.flatten()
total_pts = np.array([tup[0] for tup in pairs])
for i, (xpts, ypts) in enumerate(pairs):
    axes[i].plot(z_vals, times[0, i, :], marker='o', label="{} pts per edge".format(xpts))
    axes[i].legend(loc='best')
for ax in axes[-3:]:
    ax.set_xlabel("Z Coordinate")
for ax in axes[0::3]:
    ax.set_ylabel("Time (seconds)")
plt.tight_layout()
plt.show()

# Run Time Scaling w.r.t Sampling Points, Fixed Basis

## N = 200

In [ ]:
cols = 3
rows = int(np.ceil(len(z_vals)/cols))
fig, axes = plt.subplots(rows, cols, figsize=(16, 13), sharey=True)
axes = axes.flatten()
total_pts = np.array([tup[0] for tup in pairs])
for i, z in enumerate(z_vals):
    axes[i].plot(total_pts, times[0, :, i], marker='o', label="z = {:.3f}".format(z))
    axes[i].legend(loc='best')
for ax in axes[-3:]:
    ax.set_xlabel("Number of pts per edge")
for ax in axes[0::3]:
    ax.set_ylabel("Time (seconds)")
plt.tight_layout()
plt.show()

## N = 500

In [ ]:
cols = 3
rows = int(np.ceil(len(z_vals)/cols))
fig, axes = plt.subplots(rows, cols, figsize=(16, 13), sharey=True)
axes = axes.flatten()
total_pts = np.array([tup[0] for tup in pairs])
for i, z in enumerate(z_vals):
    axes[i].plot(total_pts, times[-1, :, i], marker='o', label="z = {:.3f}".format(z))
    axes[i].legend(loc='best')
for ax in axes[-3:]:
    ax.set_xlabel("Number of pts per edge")
for ax in axes[0::3]:
    ax.set_ylabel("Time (seconds)")
plt.tight_layout()
plt.show()

# Run Time Scaling w.r.t Sampling Points, z = 1.125

In [ ]:
cols = 3
rows = int(np.ceil(len(numbasis)/cols))
fig, axes = plt.subplots(rows, cols, figsize=(16, 13), sharex=True)
axes = axes.flatten()
total_pts = np.array([tup[0] for tup in pairs])
for i, basis in enumerate(numbasis):
    axes[i].plot(total_pts[1:], times[i, 1:, 5], marker='o', label="N = {}".format(basis))
    axes[i].legend(loc='best')
for ax in axes[-3:]:
    ax.set_xlabel("Number of pts per edge")
for ax in axes[0::3]:
    ax.set_ylabel("Run Time (seconds)")
plt.tight_layout()
plt.show()

# Run Time Scaling w.r.t Basis Terms, z=1.125

In [ ]:
cols = 3
rows = int(np.ceil(len(pairs)/cols))
fig, axes = plt.subplots(rows, cols, figsize=(16, 13))
axes = axes.flatten()
total_pts = np.array([tup[0] for tup in pairs])
for i, pair in enumerate(pairs):
    axes[i].plot(numbasis, times[:, i, 5], marker='o', label="Pts/Edge = {}".format(pair[0]))
    axes[i].legend(loc='best')
for ax in axes[-3:]:
    ax.set_xlabel("Number of Basis Terms")
for ax in axes[0::3]:
    ax.set_ylabel("Time (seconds)")
plt.tight_layout()
plt.show()

In [ ]:
from scipy.optimize import curve_fit

def line(x, m, b):
    return m*x + b

def cube(x, a, m, b):
    return a*x**m + b

cols = 3
rows = int(np.ceil(len(pairs)/cols))
fig, axes = plt.subplots(rows, cols, figsize=(16, 13))
axes = axes.flatten()
total_pts = np.array([tup[0] for tup in pairs])
x = np.log(numbasis)
for i, pair in enumerate(pairs):
    y = np.log(times[:, i, 5])
    opt_pars, par_cov = curve_fit(cube, numbasis, times[:, i, 5])
    print(opt_pars)
    print(par_cov)
    opt_pars, par_cov = curve_fit(line, x, y)
    slope = opt_pars[0]
    y_int = np.exp(opt_pars[1])
    lab = "Pts/Edge = {}\nSlope = {:.3f}".format(pair[0], slope)
    axes[i].plot(x, y, marker='o', label=lab)
    axes[i].legend(loc='best')
for ax in axes[-3:]:
    ax.set_xlabel("Number of Basis Terms")
for ax in axes[0::3]:
    ax.set_ylabel("Time (seconds)")
plt.tight_layout()
plt.show()

In [ ]:
xpts = 125
ypts = 125
conf[('Simulation', 'x_samples')] = xpts
conf[('Simulation', 'y_samples')] = ypts
sim = Simulator(conf)
sim.setup()
layers = get_layers(sim)
Ex, Ey, Ez, Hx, Hy, Hz = sim.compute_fields_on_plane(1.125, sim.xsamps, sim.ysamps)
normEsq = np.absolute(Ex)**2 + np.absolute(Ey)**2 + np.absolute(Ez)**2
print(normEsq.shape)

In [ ]:
layers = get_layers(sim)
layer_obj = [layer for layer in layers.values() if layer.start <= z <= layer.end].pop()
n_mat, k_mat = layer_obj.get_nk_matrix(freq)
x = np.linspace(0, .25, 125)
nkE = n_mat*k_mat*normEsq
fig = plt.figure()
plt.plot(x, normEsq[62, :], label=r'$|E|^2$')
plt.plot(x, nkE[62, :], label=r'$nk|E|^2$')
plt.xlabel('Microns')
plt.ylabel('(V/m)^2')
#fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 9))
#ax1.plot(x, normEsq[250, :])
#ax1.set_title(r'$|E|^2$')
#ax2.plot(x, nkE[250, :])
#ax2.set_title(r'$nk|E|^2$')
plt.legend()
plt.show()